In [59]:
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display as display
import librosa.feature
import soundfile as sf
import math 

In [60]:
noise_path = 'dataset/Musan-Data/noise'
noises = glob.glob(os.path.join(noise_path,'*/*.wav'))

music_path = 'dataset/Musan-Data/music'
music = glob.glob(os.path.join(music_path,'*/*.wav'))

speech_path = 'dataset/Musan-Data/speech'
speech = glob.glob(os.path.join(speech_path,'*/*.wav'))

In [61]:
def import_audio(filename): #Fungsi untuk ekstrak data time series dari audio
    """
    Extract audio given the filename (.wav, .flac, etc format)
    """

    audio, rate = sf.read(filename, always_2d=True)
    audio = np.reshape(audio, (1, -1))
    audio = audio[0]
    time = np.linspace(0, len(audio)/rate, len(audio), endpoint=False)
    return audio, rate, time

In [62]:
def add_noise(audio, noise, snr, rate):
    noise = noise[0:audio.shape[0]]
    
    while 1 :
        if len(audio) > len(noise):
            noise_add = noise[0:(len(audio)-len(noise))]
            noise = np.append(noise, noise_add)

        else:
            break
    audio_RMS = rmsValue(audio,len(audio))
    noise_RMS = rmsValue(noise,len(noise))
    noise_RMS_new = audio_RMS / pow(10,(snr/20))
    noise = noise*(noise_RMS_new/noise_RMS)
    snr_check = 20 * np.log10(audio_RMS/rmsValue(noise,len(noise)))

    print("noise added with SNR = " + str(round(snr_check)))

    noised = audio + noise
    noised_rate = rate

    return noised, noised_rate, round(snr_check), noise


def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
     
    for i in range(0,n):
        square += (arr[i]**2)

    mean = (square / (float)(n))
    root = math.sqrt(mean)
     
    return root
 
def pad(audio, pad_length):
    shape = np.shape(audio)
    padded_audio = np.zeros(pad_length)
    padded_audio[0:shape[0]] = audio

    return padded_audio

In [63]:
def extract_tf_mask(audio, noise, n_fft=1024, hop_length=512, win_length=1024):
    y = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
    n = librosa.stft(noise, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

    mask = (np.absolute(y)) / (np.absolute(n))
    n_frames = abs(n).shape[-1]

    return y, n, mask, n_frames

In [64]:
if os.path.exists('part')==False:
    os.mkdir('part')

In [65]:
dataset = 'dataset/Cough-Data'
types = [noises,music,speech]

X = []
y = []

a = 0
b = 0
for file in os.listdir(dataset):
    filename = os.path.join(dataset,file)
    for noise_type in types:
        print('Process ' + file + ' .....')
        num = random.choice(range(0,len(noise_type)))
        add = noise_type[num]
        
        wav,sr = librosa.load(filename,sr=None)
        duration = librosa.get_duration(wav,sr)
        if duration <= 5:
            wav = librosa.util.pad_center(wav,sr*5)

            add_wav,_ = librosa.load(add,sr=None)
            add_wav = add_wav[0:wav.shape[0]]

            snr = random.choice(range(15,25))
            mixtured,_,_,add_wav = add_noise(wav,add_wav,snr,sr)
            mixtured_tf = librosa.stft(mixtured, n_fft=1024, hop_length=512, win_length=1024)
            audio_tf, noise_tf, mask, n_frames = extract_tf_mask(wav, mixtured)

            mixtured_spec = np.absolute(mixtured_tf)
            mask_spec = mask

            X.append(mixtured_spec)
            y.append(mask_spec)

            if a%500 == 0:
                np.save('part/X_'+str(b)+'.npy',X)
                np.save('part/y_'+str(b)+'.npy',y)
                b = b + 1
                X = []
                y = []
            a = a+1

np.save('part/X_'+str(b)+'.npy',X)
np.save('part/y_'+str(b)+'.npy',y)

Process f5e4c9ed-6a33-4fb3-8969-7b434c890452.wav .....
Process f5e4c9ed-6a33-4fb3-8969-7b434c890452.wav .....
Process f5e4c9ed-6a33-4fb3-8969-7b434c890452.wav .....
Process 156d4979-9dc2-404d-ab93-d6599d12cf20.wav .....
Process 156d4979-9dc2-404d-ab93-d6599d12cf20.wav .....
Process 156d4979-9dc2-404d-ab93-d6599d12cf20.wav .....
Process NBD8pzxLyyYZs8wul5W6UUKBZbz1.wav .....
noise added with SNR = 15
Process NBD8pzxLyyYZs8wul5W6UUKBZbz1.wav .....
noise added with SNR = 17
Process NBD8pzxLyyYZs8wul5W6UUKBZbz1.wav .....
noise added with SNR = 18
Process 406d6093-d0bf-452f-8192-c6e20d160f0c.wav .....
Process 406d6093-d0bf-452f-8192-c6e20d160f0c.wav .....
Process 406d6093-d0bf-452f-8192-c6e20d160f0c.wav .....
Process 47d73557-ca2e-4660-b0d1-109e2adccf94.wav .....
Process 47d73557-ca2e-4660-b0d1-109e2adccf94.wav .....
Process 47d73557-ca2e-4660-b0d1-109e2adccf94.wav .....
Process 6df2a8e0-42a8-4c5a-bf03-15c61180f7c3.wav .....
Process 6df2a8e0-42a8-4c5a-bf03-15c61180f7c3.wav .....
Process 6df

noise added with SNR = 23
Process 4gkSRZ0IWxVAOJSuVEFhQqrzIYx1_heavy.wav .....
noise added with SNR = 22
Process 4gkSRZ0IWxVAOJSuVEFhQqrzIYx1_heavy.wav .....
noise added with SNR = 23
Process 7J57fRRIEENoMhm7xRL7HNxh7Xy2_heavy.wav .....
Process 7J57fRRIEENoMhm7xRL7HNxh7Xy2_heavy.wav .....
Process 7J57fRRIEENoMhm7xRL7HNxh7Xy2_heavy.wav .....
Process 072fdfd5-9418-4db4-88e5-8a574b40d51e.wav .....
Process 072fdfd5-9418-4db4-88e5-8a574b40d51e.wav .....
Process 072fdfd5-9418-4db4-88e5-8a574b40d51e.wav .....
Process train_195.wav .....
Process train_195.wav .....
Process train_195.wav .....
Process aa059172-1850-4afe-8809-2fe79603286a.wav .....
Process aa059172-1850-4afe-8809-2fe79603286a.wav .....
Process aa059172-1850-4afe-8809-2fe79603286a.wav .....
Process c55e4ac5-03f1-4432-ada6-be2c30e76bcb.wav .....
Process c55e4ac5-03f1-4432-ada6-be2c30e76bcb.wav .....
Process c55e4ac5-03f1-4432-ada6-be2c30e76bcb.wav .....
Process d852279e-ba08-45a1-bdb0-378aefc36b19.wav .....
Process d852279e-ba08-4

noise added with SNR = 17
Process eccf5612-6527-4a8c-95ca-59333d90eaed.wav .....
noise added with SNR = 21
Process eccf5612-6527-4a8c-95ca-59333d90eaed.wav .....
noise added with SNR = 20
Process jaRfWYxsWjV2L833wIIyuivEWFG3.wav .....
Process jaRfWYxsWjV2L833wIIyuivEWFG3.wav .....
Process jaRfWYxsWjV2L833wIIyuivEWFG3.wav .....
Process 58eac2ea-61ee-4e11-b200-7a824ae7e87f.wav .....
Process 58eac2ea-61ee-4e11-b200-7a824ae7e87f.wav .....
Process 58eac2ea-61ee-4e11-b200-7a824ae7e87f.wav .....
Process 74d00b7b-3e72-4237-b53f-6a002fb2d0f5.wav .....
Process 74d00b7b-3e72-4237-b53f-6a002fb2d0f5.wav .....
Process 74d00b7b-3e72-4237-b53f-6a002fb2d0f5.wav .....
Process c26e5c87-6f9d-41a9-b241-b3236caf0ece.wav .....
Process c26e5c87-6f9d-41a9-b241-b3236caf0ece.wav .....
Process c26e5c87-6f9d-41a9-b241-b3236caf0ece.wav .....
Process ec83e7ba-91a6-46c7-aabd-dac5597761bb.wav .....
Process ec83e7ba-91a6-46c7-aabd-dac5597761bb.wav .....
Process ec83e7ba-91a6-46c7-aabd-dac5597761bb.wav .....
Process dev

noise added with SNR = 18
Process c62a7d17-49e8-4f21-b862-230c26335e4f.wav .....
noise added with SNR = 18
Process c62a7d17-49e8-4f21-b862-230c26335e4f.wav .....
noise added with SNR = 17
Process 5abf4c0b-7b42-4adc-aaf3-c7d6e56e8ae1.wav .....
Process 5abf4c0b-7b42-4adc-aaf3-c7d6e56e8ae1.wav .....
Process 5abf4c0b-7b42-4adc-aaf3-c7d6e56e8ae1.wav .....
Process fWUMPPgD78VZinzLLLwqbyTysJa2_heavy.wav .....
noise added with SNR = 24
Process fWUMPPgD78VZinzLLLwqbyTysJa2_heavy.wav .....
noise added with SNR = 21
Process fWUMPPgD78VZinzLLLwqbyTysJa2_heavy.wav .....
noise added with SNR = 18
Process Kj5Pw7GpqlhOhtfYKysgdFZcijG3_heavy.wav .....
Process Kj5Pw7GpqlhOhtfYKysgdFZcijG3_heavy.wav .....
Process Kj5Pw7GpqlhOhtfYKysgdFZcijG3_heavy.wav .....
Process euld60aqc2OtnNoKYx9P4WDzVM92.wav .....
Process euld60aqc2OtnNoKYx9P4WDzVM92.wav .....
Process euld60aqc2OtnNoKYx9P4WDzVM92.wav .....
Process 6f4c81da-1cec-4910-998d-239b810082ae.wav .....
Process 6f4c81da-1cec-4910-998d-239b810082ae.wav .....


noise added with SNR = 20
Process train_183.wav .....
noise added with SNR = 19
Process train_183.wav .....
noise added with SNR = 23
Process 8e097f47-9222-47e0-ab3a-6f80c880d6e7.wav .....
Process 8e097f47-9222-47e0-ab3a-6f80c880d6e7.wav .....
Process 8e097f47-9222-47e0-ab3a-6f80c880d6e7.wav .....
Process 8b04c1fa-4fbe-45f3-9b4a-5cc6a450c43c.wav .....
noise added with SNR = 17
Process 8b04c1fa-4fbe-45f3-9b4a-5cc6a450c43c.wav .....
noise added with SNR = 20
Process 8b04c1fa-4fbe-45f3-9b4a-5cc6a450c43c.wav .....
noise added with SNR = 20
Process 2f42e6dd-5338-43c8-9191-1a5cb4c50d3e.wav .....
Process 2f42e6dd-5338-43c8-9191-1a5cb4c50d3e.wav .....
Process 2f42e6dd-5338-43c8-9191-1a5cb4c50d3e.wav .....
Process ef4b3fa8-7c0d-4220-b3b3-47b581d1085b.wav .....
Process ef4b3fa8-7c0d-4220-b3b3-47b581d1085b.wav .....
Process ef4b3fa8-7c0d-4220-b3b3-47b581d1085b.wav .....
Process a3121531-4ee7-48ef-baae-d59ed2c09b06.wav .....
Process a3121531-4ee7-48ef-baae-d59ed2c09b06.wav .....
Process a3121531-4

...

noise added with SNR = 15
Process JJAEXqn68kVaaOsFjX7n5Qehaws1.wav .....
noise added with SNR = 21
Process JJAEXqn68kVaaOsFjX7n5Qehaws1.wav .....
noise added with SNR = 15
Process bea2108f-1993-4ff2-afe1-8a06c5b5e766.wav .....
Process bea2108f-1993-4ff2-afe1-8a06c5b5e766.wav .....
Process bea2108f-1993-4ff2-afe1-8a06c5b5e766.wav .....
Process d0305de6-ef86-4775-9ae3-8d53d8fc7e1b.wav .....
Process d0305de6-ef86-4775-9ae3-8d53d8fc7e1b.wav .....
Process d0305de6-ef86-4775-9ae3-8d53d8fc7e1b.wav .....
Process afadbd6d-433c-4ffc-81da-1f0217bac43e.wav .....
Process afadbd6d-433c-4ffc-81da-1f0217bac43e.wav .....
Process afadbd6d-433c-4ffc-81da-1f0217bac43e.wav .....
Process dxKWK5iTjWdjch9zRGCnc7R0eeq2.wav .....
noise added with SNR = 18
Process dxKWK5iTjWdjch9zRGCnc7R0eeq2.wav .....
noise added with SNR = 15
Process dxKWK5iTjWdjch9zRGCnc7R0eeq2.wav .....
noise added with SNR = 22
Process 4ySGp3wKh0dbtEap33yRsDv3LE73_heavy.wav .....
noise added with SNR = 21
Process 4ySGp3wKh0dbtEap33yRsDv3LE7

noise added with SNR = 18
Process 814b168b-0843-4a79-a149-2a9b54da9231.wav .....
noise added with SNR = 16
Process 814b168b-0843-4a79-a149-2a9b54da9231.wav .....
noise added with SNR = 15
Process 5874af96-541d-443c-878d-996bd7bfa1cf.wav .....
Process 5874af96-541d-443c-878d-996bd7bfa1cf.wav .....
Process 5874af96-541d-443c-878d-996bd7bfa1cf.wav .....
Process 0ad50dd1-46d4-4a99-a156-2c95a1c094d8.wav .....
Process 0ad50dd1-46d4-4a99-a156-2c95a1c094d8.wav .....
Process 0ad50dd1-46d4-4a99-a156-2c95a1c094d8.wav .....
Process 0f8fb3e0-1a30-4bd3-982a-24342a0bdc70.wav .....
noise added with SNR = 19
Process 0f8fb3e0-1a30-4bd3-982a-24342a0bdc70.wav .....
noise added with SNR = 24
Process 0f8fb3e0-1a30-4bd3-982a-24342a0bdc70.wav .....
noise added with SNR = 19
Process r3sBhLDneedRBXKLvZU0PL2Q4R82_heavy.wav .....
noise added with SNR = 24
Process r3sBhLDneedRBXKLvZU0PL2Q4R82_heavy.wav .....
noise added with SNR = 20
Process r3sBhLDneedRBXKLvZU0PL2Q4R82_heavy.wav .....
noise added with SNR = 18
Pr

noise added with SNR = 18
Process qpCGgfO05mWt67NzsFBKWJ5ZF1U2_heavy.wav .....
Process qpCGgfO05mWt67NzsFBKWJ5ZF1U2_heavy.wav .....
Process qpCGgfO05mWt67NzsFBKWJ5ZF1U2_heavy.wav .....
Process 851d3141-589e-4a63-a81f-837b85585ee7.wav .....
Process 851d3141-589e-4a63-a81f-837b85585ee7.wav .....
Process 851d3141-589e-4a63-a81f-837b85585ee7.wav .....
Process b4c628c8-33ca-495b-a614-74ccadc1e8c7.wav .....
Process b4c628c8-33ca-495b-a614-74ccadc1e8c7.wav .....
Process b4c628c8-33ca-495b-a614-74ccadc1e8c7.wav .....
Process 230d5eb1-913e-4150-9678-75f8b7c9d319.wav .....
Process 230d5eb1-913e-4150-9678-75f8b7c9d319.wav .....
Process 230d5eb1-913e-4150-9678-75f8b7c9d319.wav .....
Process 0379c586-c500-483c-83a6-95b63afe6931.wav .....
Process 0379c586-c500-483c-83a6-95b63afe6931.wav .....
Process 0379c586-c500-483c-83a6-95b63afe6931.wav .....
Process 433ecbf4-f30c-4252-bb8c-072bb13f07ab.wav .....
Process 433ecbf4-f30c-4252-bb8c-072bb13f07ab.wav .....
Process 433ecbf4-f30c-4252-bb8c-072bb13f07ab.

noise added with SNR = 22
Process 1bf98b07-4729-4878-8d8d-e1422c4531a9.wav .....
noise added with SNR = 15
Process 1bf98b07-4729-4878-8d8d-e1422c4531a9.wav .....
noise added with SNR = 20
Process test_140.wav .....
Process test_140.wav .....
Process test_140.wav .....
Process 382a7410-18f2-449a-b57f-d2d625bee787.wav .....
Process 382a7410-18f2-449a-b57f-d2d625bee787.wav .....
Process 382a7410-18f2-449a-b57f-d2d625bee787.wav .....
Process 4f4588e4-7105-4730-b541-3ce45c6e6237.wav .....
Process 4f4588e4-7105-4730-b541-3ce45c6e6237.wav .....
Process 4f4588e4-7105-4730-b541-3ce45c6e6237.wav .....
Process 0b2f37f5-0b2d-402f-a28e-6e80331e44a5.wav .....
Process 0b2f37f5-0b2d-402f-a28e-6e80331e44a5.wav .....
Process 0b2f37f5-0b2d-402f-a28e-6e80331e44a5.wav .....
Process fuZ1Iu9k84gcTbZKIWElwq2RsLo1.wav .....
Process fuZ1Iu9k84gcTbZKIWElwq2RsLo1.wav .....
Process fuZ1Iu9k84gcTbZKIWElwq2RsLo1.wav .....
Process 0044cb7b-448c-44e5-8302-ad8bd106fe3e.wav .....
Process 0044cb7b-448c-44e5-8302-ad8bd106

noise added with SNR = 19
Process aB7zSIAFW7bCmQC7IBUWFjNncTF3.wav .....
noise added with SNR = 19
Process aB7zSIAFW7bCmQC7IBUWFjNncTF3.wav .....
noise added with SNR = 18
Process 941ea4ab-a733-4bf0-ab8e-1b271eb04aaf.wav .....
noise added with SNR = 22
Process 941ea4ab-a733-4bf0-ab8e-1b271eb04aaf.wav .....
noise added with SNR = 15
Process 941ea4ab-a733-4bf0-ab8e-1b271eb04aaf.wav .....
noise added with SNR = 24
Process 6bf12675-e71e-4a6e-b77b-59c95edc52ab.wav .....
noise added with SNR = 19
Process 6bf12675-e71e-4a6e-b77b-59c95edc52ab.wav .....
noise added with SNR = 21
Process 6bf12675-e71e-4a6e-b77b-59c95edc52ab.wav .....
noise added with SNR = 19
Process 3FD9salx6KQf114qJSSvW74Ffi63.wav .....
Process 3FD9salx6KQf114qJSSvW74Ffi63.wav .....
Process 3FD9salx6KQf114qJSSvW74Ffi63.wav .....
Process 01b83641-dc8e-4231-a50e-32ede1ffd5e2.wav .....
Process 01b83641-dc8e-4231-a50e-32ede1ffd5e2.wav .....
Process 01b83641-dc8e-4231-a50e-32ede1ffd5e2.wav .....
Process c76dcf90-f920-4ea2-ba15-2542

In [66]:
X = []
y = []

for i in range(0,4):
    X_load = np.load('part/X_'+str(i)+'.npy')
    y_load = np.load('part/y_'+str(i)+'.npy')

    X.append(X_load)
    y.append(y_load)

In [67]:
X = np.concatenate(X)
y = np.concatenate(y)

np.save('X.npy',X)
np.save('y.npy',y)

In [68]:
X = np.load('X.npy')
y = np.load('y.npy')

In [69]:
X = X.reshape(X.shape[0],1,X.shape[1],X.shape[2])
y = y.reshape(y.shape[0],1,y.shape[1],y.shape[2])

In [70]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils import rnn
import time

In [71]:
cuda = torch.device('cuda')

In [72]:
X = torch.from_numpy(X).cuda()
y = torch.from_numpy(y).cuda()

In [73]:
class ReverbDataset(Dataset):
    """
    Reverberation dataset
    """

    def __init__(self, X, y):
        """
        X: (# examples, 1, 128, 340) tensor containing reverberant spectrograms
        y: (# examples, 1, 128, 340) tensor containing target spectrograms
        """

        self.X = X
        self.y = y
        #self.z = z

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]#, self.z[index]

In [74]:
length = len(X)
split = int(length*0.8)

In [75]:
dataset_train = ReverbDataset(X[0:split,:,:,:], y[0:split,:,:,:])
dataset_val = ReverbDataset(X[split:length,:,:,:], y[split:length,:,:,:])

In [76]:
train_loader = DataLoader(dataset_train, batch_size = 1, shuffle = True, num_workers = 0, pin_memory = False)
val_loader = DataLoader(dataset_val, batch_size = 1, shuffle = True, num_workers = 0, pin_memory = False)

In [77]:
class NNmodel(nn.Module):
    def __init__(self, num_layers=1, dropout=0.1, rescale=False, bidirectional=True):
        super(NNmodel, self).__init__()
        
        num_directions = 2 if bidirectional else 1
        self.outfc = nn.Linear(num_directions * 1024, 513, bias=False)
        
        self.dropout = nn.Dropout(dropout)

        self.rnn = nn.LSTM(
            input_size = 513,
            hidden_size = 1024,
            num_layers = num_layers,
            bidirectional = bidirectional
        )


    def forward(self, x):
        #print(x.shape)
        
        x = x.reshape(x.shape[3], 1, x.shape[2])
        #print(x.shape)
        
        x, _ = self.rnn(x)
        #print(x.shape)

        x = self.outfc(x[:, 0, :])
        #print(x.shape)

        x = x.reshape(x.shape[1],x.shape[0])
        #print(x.shape)

        return x

model = NNmodel(num_layers=1,
               dropout=0.5,
               rescale=False,
               bidirectional=True)

print(model)

NNmodel(
  (outfc): Linear(in_features=2048, out_features=513, bias=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): LSTM(513, 1024, bidirectional=True)
)


In [78]:
def trainer_deg(model, train_loader, val_loader, checkpoints, nEpochs=100, lr=1e-3):
    
    criterion = nn.CrossEntropyLoss()
    beta1 = 0.9
    beta2 = 0.999
    lr_decay = 0.97
    decay_rate = 2
    
    optimizer = torch.optim.Adam(model.parameters(), lr, (beta1, beta2), eps=1e-08)
    
    train_loss = []
    val_loss = []
    
    model.train()
    
    epoch_time = []


    for epoch in range(nEpochs):
        
        start_time = time.process_time()
        
        temp_train_loss = 0.0
        corrects_train = 0
        counter = 0
        epoch_times = []
        
        forward_time = []
        backward_time = []

        for i, (rev_data, female_data) in enumerate(train_loader):
            
            counter += 1
        
            optimizer.zero_grad()
        
            for_start_time = time.process_time()
            output = model(rev_data)
            for_end_time = time.process_time()
            
            forward_time.append(for_end_time-for_start_time)
            
            loss_train = criterion(output, female_data[0,0])
            
            back_start_time = time.process_time()
            loss_train.backward()
            back_end_time = time.process_time()

            backward_time.append(back_end_time-back_start_time)
            
            optimizer.step()
            temp_train_loss += loss_train.item()
        
            #print(counter)
            #print("\n")
            if counter%200 == 0:
                print("Epoch {} -- Step: {}/{} || Average Loss for Epoch: {}".format(epoch+1, counter, len(train_loader), temp_train_loss/counter))
        
        current_time =  time.process_time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch+1, nEpochs, temp_train_loss/len(train_loader)))
        print("Total Time Elapsed: {} seconds\n".format(str(current_time-start_time)))
    
        epoch_times.append(current_time-start_time)
        
        #Validation
        temp_val_loss = 0.0 
        model.eval()
        start_time_val = time.process_time()
        
        with torch.no_grad():
            for i, (rev_data, female_data) in enumerate(val_loader):

                output = model(rev_data)
            
                loss_val = criterion(output, female_data[0,0])
                temp_val_loss += loss_val.item()
        
        print("Evaluation Time: {}".format(str(time.process_time()-start_time_val)))
        print('Epoch : {} || Train Loss: {:.3f} || Val Loss: {:.3f} \n'\
         .format(epoch+1, loss_train.item(), loss_val.item()))
    
        train_loss.append(temp_train_loss)
        val_loss.append(loss_val.item())
        
        model.train()
      
        if epoch%5==0:
            torch.save(model.state_dict(), checkpoints[0])
            torch.save(train_loss, checkpoints[1])
            torch.save(val_loss, checkpoints[2])
            print("saved models")
        
        if loss_val.item() <= min(val_loss):
            torch.save(model.state_dict(), checkpoints[3])
            print('saved models best')


        if (epoch % decay_rate == 1):
            optimizer.param_groups[0]['lr'] *= lr_decay
        
    print('Best Loss Val: '+ str(min(val_loss)))
    
    return train_loss, val_loss, forward_time, backward_time

In [79]:
if os.path.exists('ckpt')==False:
    os.mkdir('ckpt')

In [80]:
if torch.cuda.is_available():
    model.cuda()

In [81]:
checkpoints = ['ckpt/testing.pth', 'ckpt/train_loss_testing.pth', 
               'ckpt/val_loss_testing.pth','ckpt/testing_best.pth']

train_loss, val_loss, forward, backward = trainer_deg(model, train_loader, val_loader, checkpoints, nEpochs = 100, lr=1e-3)

Epoch 1 -- Step: 200/974 || Average Loss for Epoch: 337.1142127990723
Epoch 1 -- Step: 400/974 || Average Loss for Epoch: 343.586615524292
Epoch 1 -- Step: 600/974 || Average Loss for Epoch: 344.36819922129314
Epoch 1 -- Step: 800/974 || Average Loss for Epoch: 342.16261891365053
Epoch 1/100 Done, Total Loss: 339.4648172899438
Total Time Elapsed: 34.150088014000175 seconds

Evaluation Time: 3.550333529999989
Epoch : 1 || Train Loss: 365.673 || Val Loss: 191.647 

saved models
saved models best
Epoch 2 -- Step: 200/974 || Average Loss for Epoch: 326.9796315956116
Epoch 2 -- Step: 400/974 || Average Loss for Epoch: 325.59417885780334
Epoch 2 -- Step: 600/974 || Average Loss for Epoch: 333.6415562438965
Epoch 2 -- Step: 800/974 || Average Loss for Epoch: 333.47785695552824
Epoch 2/100 Done, Total Loss: 337.30037593254076
Total Time Elapsed: 34.20362241800012 seconds

Evaluation Time: 3.5611119160003
Epoch : 2 || Train Loss: 437.850 || Val Loss: 213.836 

Epoch 3 -- Step: 200/974 || Averag

Epoch 18 -- Step: 400/974 || Average Loss for Epoch: 319.90368465423586
Epoch 18 -- Step: 600/974 || Average Loss for Epoch: 313.2651728057861
Epoch 18 -- Step: 800/974 || Average Loss for Epoch: 318.39437495708466
Epoch 18/100 Done, Total Loss: 319.87063160373447
Total Time Elapsed: 34.22428572700028 seconds

Evaluation Time: 3.556913651999821
Epoch : 18 || Train Loss: 367.234 || Val Loss: 335.873 

Epoch 19 -- Step: 200/974 || Average Loss for Epoch: 316.13393491744995
Epoch 19 -- Step: 400/974 || Average Loss for Epoch: 320.26189847946165
Epoch 19 -- Step: 600/974 || Average Loss for Epoch: 321.203777662913
Epoch 19 -- Step: 800/974 || Average Loss for Epoch: 323.0843139600754
Epoch 19/100 Done, Total Loss: 319.3218226070521
Total Time Elapsed: 34.19400295200012 seconds

Evaluation Time: 3.5592622829999527
Epoch : 19 || Train Loss: 517.769 || Val Loss: 339.903 

Epoch 20 -- Step: 200/974 || Average Loss for Epoch: 313.7216767692566
Epoch 20 -- Step: 400/974 || Average Loss for Epoch

Epoch 35 -- Step: 600/974 || Average Loss for Epoch: 314.02438045183817
Epoch 35 -- Step: 800/974 || Average Loss for Epoch: 312.414444463253
Epoch 35/100 Done, Total Loss: 314.55297344959735
Total Time Elapsed: 34.24910826799987 seconds

Evaluation Time: 3.559651161999682
Epoch : 35 || Train Loss: 235.978 || Val Loss: 537.561 

Epoch 36 -- Step: 200/974 || Average Loss for Epoch: 342.6590071296692
Epoch 36 -- Step: 400/974 || Average Loss for Epoch: 323.10652586936953
Epoch 36 -- Step: 600/974 || Average Loss for Epoch: 316.9572917524974
Epoch 36 -- Step: 800/974 || Average Loss for Epoch: 315.04663193941116
Epoch 36/100 Done, Total Loss: 314.3986783957824
Total Time Elapsed: 34.212019550999685 seconds

Evaluation Time: 3.562607226000182
Epoch : 36 || Train Loss: 375.573 || Val Loss: 177.545 

saved models
Epoch 37 -- Step: 200/974 || Average Loss for Epoch: 323.1298861789703
Epoch 37 -- Step: 400/974 || Average Loss for Epoch: 322.24415400028226
Epoch 37 -- Step: 600/974 || Average L

Epoch 52 -- Step: 800/974 || Average Loss for Epoch: 308.8376351475716
Epoch 52/100 Done, Total Loss: 312.0388625319244
Total Time Elapsed: 34.21347004099971 seconds

Evaluation Time: 3.562380653999753
Epoch : 52 || Train Loss: 225.137 || Val Loss: 590.164 

Epoch 53 -- Step: 200/974 || Average Loss for Epoch: 324.83538333892824
Epoch 53 -- Step: 400/974 || Average Loss for Epoch: 314.84891377449037
Epoch 53 -- Step: 600/974 || Average Loss for Epoch: 313.3860155105591
Epoch 53 -- Step: 800/974 || Average Loss for Epoch: 312.5260404753685
Epoch 53/100 Done, Total Loss: 311.93067842244614
Total Time Elapsed: 34.230518408000535 seconds

Evaluation Time: 3.561121424999328
Epoch : 53 || Train Loss: 207.876 || Val Loss: 262.668 

Epoch 54 -- Step: 200/974 || Average Loss for Epoch: 300.1760508060455
Epoch 54 -- Step: 400/974 || Average Loss for Epoch: 314.3047458600998
Epoch 54 -- Step: 600/974 || Average Loss for Epoch: 310.90073723157246
Epoch 54 -- Step: 800/974 || Average Loss for Epoch

Evaluation Time: 3.5630585110002357
Epoch : 69 || Train Loss: 485.847 || Val Loss: 210.301 

Epoch 70 -- Step: 200/974 || Average Loss for Epoch: 299.1245723152161
Epoch 70 -- Step: 400/974 || Average Loss for Epoch: 295.5921152114868
Epoch 70 -- Step: 600/974 || Average Loss for Epoch: 301.06602358500163
Epoch 70 -- Step: 800/974 || Average Loss for Epoch: 308.5006913232803
Epoch 70/100 Done, Total Loss: 310.5029889328034
Total Time Elapsed: 34.27699258800021 seconds

Evaluation Time: 3.564496340000005
Epoch : 70 || Train Loss: 66.221 || Val Loss: 479.872 

Epoch 71 -- Step: 200/974 || Average Loss for Epoch: 313.162120513916
Epoch 71 -- Step: 400/974 || Average Loss for Epoch: 311.22093052864074
Epoch 71 -- Step: 600/974 || Average Loss for Epoch: 311.99117436726885
Epoch 71 -- Step: 800/974 || Average Loss for Epoch: 309.9263144779205
Epoch 71/100 Done, Total Loss: 310.43962818592234
Total Time Elapsed: 34.211498359999496 seconds

Evaluation Time: 3.5614314779995766
Epoch : 71 || Tr

saved models
Epoch 87 -- Step: 200/974 || Average Loss for Epoch: 297.52179243087767
Epoch 87 -- Step: 400/974 || Average Loss for Epoch: 310.47775860786436
Epoch 87 -- Step: 600/974 || Average Loss for Epoch: 303.62240765253705
Epoch 87 -- Step: 800/974 || Average Loss for Epoch: 306.2265087032318
Epoch 87/100 Done, Total Loss: 309.57978172615566
Total Time Elapsed: 34.27338783399864 seconds

Evaluation Time: 3.5609745839992684
Epoch : 87 || Train Loss: 707.099 || Val Loss: 153.589 

Epoch 88 -- Step: 200/974 || Average Loss for Epoch: 319.91951726913453
Epoch 88 -- Step: 400/974 || Average Loss for Epoch: 315.50802737236023
Epoch 88 -- Step: 600/974 || Average Loss for Epoch: 314.7040709495544
Epoch 88 -- Step: 800/974 || Average Loss for Epoch: 313.3210409450531
Epoch 88/100 Done, Total Loss: 309.5558730718781
Total Time Elapsed: 34.27226883399999 seconds

Evaluation Time: 3.560241209999731
Epoch : 88 || Train Loss: 215.383 || Val Loss: 211.446 

Epoch 89 -- Step: 200/974 || Average

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import os
import sys
sys.path.append(os.path.abspath('src'))
from segmentation import segment_cough, compute_SNR
from feature_class import features
from DSP import classify_cough
from scipy.io import wavfile
import pickle

In [84]:
loaded_model = pickle.load(open(os.path.join('models', 'cough_classifier'), 'rb'))
loaded_scaler = pickle.load(open(os.path.join('models','cough_classification_scaler'), 'rb'))

In [85]:
model.load_state_dict(torch.load('ckpt/testing_best.pth'))

<All keys matched successfully>

In [86]:
pred_model = model
if torch.cuda.is_available():
    pred_model.cuda()

In [87]:
data = 'classification/other/test_001.wav'

In [88]:
wav,sr = librosa.load(data, sr=None)
wav = librosa.util.pad_center(wav, 5*sr)

In [89]:
audio_stft = librosa.stft(wav, n_fft=1024, hop_length=512, win_length=1024)
audio_stft = audio_stft.reshape(1,1,audio_stft.shape[0],audio_stft.shape[1])
audio_test = torch.cuda.FloatTensor(np.absolute(audio_stft))

In [90]:
prediction = pred_model((audio_test))

In [91]:
mask_pred = prediction.cpu().detach().numpy()

In [92]:
audio_stft = audio_stft.reshape(audio_stft.shape[2],audio_stft.shape[3])

In [93]:
from IPython.display import Audio

In [94]:
audio_pred = (audio_stft) * (mask_pred)
audio = librosa.istft(audio_pred, hop_length=512, win_length=1024)
audio = audio/max(np.absolute(audio))
audio,_ = librosa.effects.trim(audio)

snr = compute_SNR(audio,sr)
print("The SNR of the cough signal is {0}".format(snr))

probability = classify_cough(audio, sr, loaded_model, loaded_scaler)
value = round(probability*100,2)
print("The file has a {0}\% probability of being a cough".format(value))

Audio(audio, rate=sr)

The SNR of the cough signal is 7.876421213150024
The file has a 4.36\% probability of being a cough


In [57]:
wav,_ = librosa.effects.trim(wav)

snr = compute_SNR(wav,sr)
print("The SNR of the cough signal is {0}".format(snr))

probability = classify_cough(wav, sr, loaded_model, loaded_scaler)
value = round(probability*100,2)
print("The file has a {0}\% probability of being a cough".format(value))

Audio(wav, rate=sr)

The SNR of the cough signal is 4.70611572265625
The file has a 8.13\% probability of being a cough


In [96]:
data = 'classification/other/'
for i in os.listdir(data):
    filename = os.path.join(data,i)
    wav_audio,sr = librosa.load(filename, sr=None)
    duration = librosa.get_duration(wav_audio,sr)
    if duration <= 5 :
        wav = librosa.util.pad_center(wav_audio, 5*sr)
        audio_stft =  librosa.stft(wav, n_fft=1024, hop_length=512, win_length=1024)
        audio_stft = audio_stft.reshape(1,1,audio_stft.shape[0],audio_stft.shape[1])
        audio_test = torch.cuda.FloatTensor(np.absolute(audio_stft))

        prediction = pred_model(audio_test)

        mask_pred = prediction.cpu().detach().numpy()
        audio_stft = audio_stft.reshape(audio_stft.shape[2],audio_stft.shape[3])

        audio_pred = audio_stft * (mask_pred)
        audio = librosa.istft(audio_pred, hop_length=512, win_length=1024)
        audio = audio/max(np.absolute(audio))
        audio,_ = librosa.effects.trim(audio)
        wav,_ = librosa.effects.trim(wav)

        print('='*40)
        snr_wav = compute_SNR(wav,sr)
        snr_audio = compute_SNR(audio,sr)
        print("The SNR of {0} is {1} and after noise masking is {2}".format(i,snr_wav,snr_audio))

        probability = classify_cough(wav_audio, sr, loaded_model, loaded_scaler)
        value = round(probability*100,2)
        print("The file has a {0}\% probability of being a cough before masking".format(value))

        probability = classify_cough(audio, sr, loaded_model, loaded_scaler)
        value = round(probability*100,2)
        print("The file has a {0}\% probability of being a cough after masking".format(value))

The SNR of train_254.wav is 12.729378938674927 and after noise masking is 18.00622582435608
The file has a 80.86\% probability of being a cough before masking
The file has a 41.13\% probability of being a cough after masking
The SNR of test_050.wav is 27.557930946350098 and after noise masking is 32.519400119781494
The file has a 26.04\% probability of being a cough before masking
The file has a 24.09\% probability of being a cough after masking
The SNR of test_200.wav is 26.6363525390625 and after noise masking is 3.35174024105072
The file has a 58.47\% probability of being a cough before masking
The file has a 28.95\% probability of being a cough after masking
The SNR of devel_113.wav is 0 and after noise masking is 0
The file has a 74.67\% probability of being a cough before masking
The file has a 60.66\% probability of being a cough after masking
The SNR of train_073.wav is -0.8574759215116501 and after noise masking is 0
The file has a 17.41\% probability of being a cough before m

The SNR of devel_217.wav is 8.635870218276978 and after noise masking is 8.799546957015991
The file has a 9.35\% probability of being a cough before masking
The file has a 12.01\% probability of being a cough after masking
The SNR of test_078.wav is 17.118444442749023 and after noise masking is 19.564555883407593
The file has a 45.64\% probability of being a cough before masking
The file has a 68.79\% probability of being a cough after masking
The SNR of devel_197.wav is 7.752323150634766 and after noise masking is 18.784778118133545
The file has a 56.44\% probability of being a cough before masking
The file has a 84.84\% probability of being a cough after masking
The SNR of train_189.wav is 6.626545190811157 and after noise masking is 0
The file has a 54.56\% probability of being a cough before masking
The file has a 23.81\% probability of being a cough after masking
The SNR of devel_196.wav is 27.64763355255127 and after noise masking is 22.007453441619873
The file has a 45.94\% prob

The SNR of test_162.wav is 2.8154173493385315 and after noise masking is 0
The file has a 61.35\% probability of being a cough before masking
The file has a 81.23\% probability of being a cough after masking
The SNR of train_247.wav is 0 and after noise masking is 0
The file has a 58.4\% probability of being a cough before masking
The file has a 33.29\% probability of being a cough after masking
The SNR of devel_128.wav is 17.704179286956787 and after noise masking is 15.530233383178711
The file has a 81.37\% probability of being a cough before masking
The file has a 70.08\% probability of being a cough after masking
The SNR of train_037.wav is 0 and after noise masking is 0
The file has a 81.14\% probability of being a cough before masking
The file has a 30.12\% probability of being a cough after masking
The SNR of devel_056.wav is 9.350905418395996 and after noise masking is 10.216721296310425
The file has a 24.22\% probability of being a cough before masking
The file has a 7.29\% pr

The SNR of devel_160.wav is 15.42839527130127 and after noise masking is 15.076842308044434
The file has a 79.93\% probability of being a cough before masking
The file has a 78.02\% probability of being a cough after masking
The SNR of devel_164.wav is 25.44051170349121 and after noise masking is 23.91103982925415
The file has a 56.4\% probability of being a cough before masking
The file has a 83.44\% probability of being a cough after masking
The SNR of devel_090.wav is 0 and after noise masking is 0
The file has a 15.16\% probability of being a cough before masking
The file has a 24.97\% probability of being a cough after masking
The SNR of train_121.wav is 9.026709794998169 and after noise masking is 8.152068257331848
The file has a 73.66\% probability of being a cough before masking
The file has a 25.9\% probability of being a cough after masking
The SNR of train_272.wav is 0 and after noise masking is 0
The file has a 3.93\% probability of being a cough before masking
The file has

The SNR of train_194.wav is 5.8137208223342896 and after noise masking is 0
The file has a 80.97\% probability of being a cough before masking
The file has a 28.5\% probability of being a cough after masking
The SNR of train_106.wav is 13.464425802230835 and after noise masking is 15.734477043151855
The file has a 30.77\% probability of being a cough before masking
The file has a 36.06\% probability of being a cough after masking
The SNR of test_080.wav is 23.05154800415039 and after noise masking is 12.319658994674683
The file has a 44.72\% probability of being a cough before masking
The file has a 14.25\% probability of being a cough after masking
The SNR of devel_087.wav is 24.54841136932373 and after noise masking is 7.978478670120239
The file has a 36.46\% probability of being a cough before masking
The file has a 30.08\% probability of being a cough after masking
The SNR of devel_069.wav is 0 and after noise masking is 0
The file has a 82.78\% probability of being a cough before 

The SNR of train_071.wav is 37.57986307144165 and after noise masking is 32.10750341415405
The file has a 94.22\% probability of being a cough before masking
The file has a 76.62\% probability of being a cough after masking
The SNR of test_001.wav is 4.70611572265625 and after noise masking is 7.876421213150024
The file has a 10.15\% probability of being a cough before masking
The file has a 4.36\% probability of being a cough after masking
The SNR of train_265.wav is 7.453089952468872 and after noise masking is 10.10601282119751
The file has a 88.07\% probability of being a cough before masking
The file has a 52.65\% probability of being a cough after masking
The SNR of train_143.wav is 31.80840253829956 and after noise masking is 8.365120887756348
The file has a 90.85\% probability of being a cough before masking
The file has a 91.31\% probability of being a cough after masking
The SNR of test_097.wav is 32.634806632995605 and after noise masking is 12.825117111206055
The file has a 

The SNR of devel_214.wav is 7.123748064041138 and after noise masking is 8.103893399238586
The file has a 93.47\% probability of being a cough before masking
The file has a 87.6\% probability of being a cough after masking
The SNR of test_203.wav is 9.516460299491882 and after noise masking is 4.0930551290512085
The file has a 7.98\% probability of being a cough before masking
The file has a 15.38\% probability of being a cough after masking
The SNR of devel_071.wav is 9.329739809036255 and after noise masking is 0
The file has a 66.44\% probability of being a cough before masking
The file has a 55.65\% probability of being a cough after masking
The SNR of test_193.wav is 9.02333915233612 and after noise masking is 9.580216407775879
The file has a 11.45\% probability of being a cough before masking
The file has a 19.49\% probability of being a cough after masking
The SNR of devel_120.wav is 6.715008020401001 and after noise masking is 0
The file has a 77.98\% probability of being a cou